## Import libraries
Here, we import the libraries required to develop our neural network model. In addition to familiar libraries such as sklearn, nltk, we also import pytorch, which is a deep learning library used for applications such as computer vision and natural language processing. To utilize the BERT model, we import the predefined models from transformers.

In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import LabelEncoder
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup
import pandas as pd
from sklearn.metrics import classification_report, accuracy_score
import io, os

## Upload data files and load data from csv to pandas dataframe


Here, you could upload the training and test data files from your local machine to Google Colab.

In [ ]:
# Upload training data file - note that it would take about 40 seconds for colab to upload your file successfully; only upload it once, otherwise you'll need to delete the previous version on the left side (file folder)
from google.colab import files
uploaded = files.upload()

In [ ]:
# Upload test data file - only upload it once, otherwise you'll need to delete the previous version on the left side (file folder)
from google.colab import files
uploaded = files.upload()


In [ ]:
df_train = pd.read_csv("train.txt", delimiter = '\t')
df_test = pd.read_csv("test.txt", delimiter= '\t')

df_train['Tweet'] = df_train['Tweet'].str.lower()
df_test['Tweet'] = df_test['Tweet'].str.lower()
df_train["Intensity Class"] = df_train["Intensity Class"].apply(lambda x: int(x.split(':')[0]))
df_train['Intensity Class'] = df_train['Intensity Class'].replace({-3: "negative", -2: "negative", -1: "neutral", 0: "neutral", 1: "neutral", 2: "positive", 3: "positive"})
df_test["Intensity Class"] = df_test["Intensity Class"].apply(lambda x: int(x.split(':')[0]))
df_test['Intensity Class'] = df_test['Intensity Class'].replace({-3: "negative", -2: "negative", -1: "neutral", 0: "neutral", 1: "neutral", 2: "positive", 3: "positive"})

## Check if GPU is available to run the neural network

In [ ]:
# Use cuda if present

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device available for running: ")
print(device)

## Section 1: Load the pretrained model and tokenize input text

In [ ]:
# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)
model.to(device)

In [ ]:
# Tokenize the input texts
max_length = 512
train_texts = df_train['Tweet'].tolist()
test_texts = df_test['Tweet'].tolist()
train_labels = df_train['Intensity Class'].tolist()
test_labels = df_test['Intensity Class'].tolist()

# Fit the encoder on the combined labels
label_encoder = LabelEncoder()
label_encoder.fit(train_labels + test_labels)  # Fit on both train and test labels


# Transform train and test labels separately
encoded_labels_train = label_encoder.transform(train_labels)
encoded_labels_test = label_encoder.transform(test_labels)

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=max_length)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=max_length)


## Section 2: Build up dataloader

In [ ]:
# Create PyTorch datasets
train_dataset = TensorDataset(torch.tensor(train_encodings['input_ids']),
                              torch.tensor(train_encodings['attention_mask']),
                              torch.tensor(encoded_labels_train))
test_dataset = TensorDataset(torch.tensor(test_encodings['input_ids']),
                             torch.tensor(test_encodings['attention_mask']),
                             torch.tensor(encoded_labels_test))

# Create DataLoader for batching and shuffling
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)


## Section 3: Build up optimizer

In [ ]:
from tqdm import tqdm
num_epochs = 5

# Define optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=5e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=100, num_training_steps=len(train_loader) * 5)

## Section 4: Train model

In [ ]:
def evaluate(model, test_loader, device):
    model.eval()
    predictions = []
    true_labels = []
    for batch in test_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=1)
            predictions.extend(preds.cpu().numpy())
            true_labels.extend(labels.cpu().numpy())

    # Calculate accuracy
    accuracy = accuracy_score(true_labels, predictions)
    return true_labels, predictions, accuracy


In [ ]:
# The training process will take 7 minutes on T4 (5 epochs).
for epoch in range(num_epochs):
    print(f'Epoch {epoch + 1}/{num_epochs}')
    epoch_loss = 0.0
    progress_bar = tqdm(enumerate(train_loader), total=len(train_loader), leave=False)
    for step, batch in progress_bar:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        scheduler.step()

        epoch_loss += loss.item()
        progress_bar.set_postfix({'loss': epoch_loss / (step + 1)})

    print(f'Training loss: {epoch_loss / len(train_loader)}')

    true_labels, predictions, accuracy = evaluate(model, test_loader, device)
    print(f'Test accuracy: {accuracy}')
    print(classification_report(true_labels, predictions, target_names=["negative", "neutral", "positive"]))


## Section 5: Run Inference and Classification Report

In [ ]:
# Define function
def predict(input_text, max_len=512):
  # Tokenize input text
  tokenized_text = tokenizer.tokenize(input_text)

  # Convert tokens to input IDs
  input_ids = tokenizer.convert_tokens_to_ids(tokenized_text)

  # Pad sequence to the same length
  padded_input_ids = input_ids + [0]*(max_len-len(input_ids))
  padded_input_ids_tensor = torch.tensor([padded_input_ids]).to(device)

  # Construct attention mask
  attention_mask = torch.tensor([[1] * len(input_ids) + [0] * (max_len-len(input_ids))]).to(device)

  # Make prediction
  model.eval()
  with torch.no_grad():
      outputs = model(padded_input_ids_tensor, attention_mask=attention_mask)
      logits = outputs.logits
      probabilities = torch.softmax(logits, dim=1)
      predicted_class = torch.argmax(probabilities, dim=1).item()
  logits = torch.softmax(logits, dim=1).squeeze().tolist()
  print(f"This review is {logits[0] * 100:.5f}% Negative;  {logits[1] * 100:.5f}% Neutral;  {logits[2] * 100:.5f}% Positive.")

predict("CSGO matchmaking is so full of closet hacking, it's a truly awful game.")
predict("The things I would do for a @nvidia 3090... unspeakable! 🧐")
predict("@mcmalveiro Hi Miguel, that's awesome! Thank you very much for updating us! 😄 -Claire")


## Exporting your results
- Download your notebook with _File -> Download .ipynb_
